#### 符合python风格的对象
* .[显示](#显示)
    - [repr](#repr)
    - [str](#str)
    - [format](#format)
* .[用bytes来重建](#用bytes来重建)

In [1]:
from array import array
import math

In [2]:
class Array2D:
    typecode = 'd'
    __slots__ = ('__x', '__y', '__weakref__', '__index') # other attributes are not allowed
    
    def __init__(self, x, y):
        self.__x = float(x)
        self.__y = float(y)

    @property
    def x(self):
        # user property to avoid redefining self.__x to ensure the instance is hashable
        return self.__x
    
    @property
    def y(self):
        return self.__y

    def __eq__(self, other):
        if self.x == other.x and self.y == other.y:
            return True
        return False

    def __hash__(self):
        return hash(self.x) ^ hash(self.y)

    def __iter__(self):
        self.__index = 0
#         return (i for i in (self.x, self.y))
        yield self.x; yield self.y
    
    def __next__(self):
        self.__index += 1
        if self.__index == 1:
            return self.x 
        elif self.__index == 2:
            return self.y
        else:
            raise StopIteration('There are only two elements')

    def __str__(self):
        return '({!r}, {!r})'.format(*self)
    
    def __repr__(self):
        # 如果fmt_spec中不指定0, 1, 2会报
        # ValueError: cannot switch from automatic field numbering to manual field specification
        class_name = type(self).__name__
        return '{0:}({1:0.3f}, {2:0.3f})'.format(class_name, *self)  
    
    def __bytes__(self):
        return (bytes([ord(self.typecode)])) + bytes(array(self.typecode, self))
    
    def __format__(self, fmt_spec: str):
        # polar coordination
        if fmt_spec.endswith('p'): 
            outer_fmt = '<{}, {}>'
            fmt_spec = fmt_spec[:-1]
            coords = (abs(self), self.angle())
        else:
            outer_fmt = '({}, {})'
            coords = self
        components = (format(c, fmt_spec) for c in coords)
        return outer_fmt.format(*components)
    
    def __abs__(self):
        ret = 0
        for i in self:
            ret += i ** 2
        return math.sqrt(ret)
    
    def angle(self):
        return math.atan2(self.y, self.x)
    
    def __bool__(self):
        if abs(self) == 0:
            return False 
        return True

    @classmethod
    def frombytes(cls, bytes):
        typecode = chr(bytes[0])
        memv = memoryview(bytes[1:]).cast(typecode)
        return cls(*memv)

In [3]:
v2 = Array2D(3, 4)


#### 显示
  


##### \_\_repr\_\_   <a name='repr'></a>
开发者调试用

In [4]:
v2 # 调用__repr__

Array2D(3.000, 4.000)

##### \_\_str\_\_   <a id='str'></a>
输出用

In [5]:
print(v2) # 调用__str__

(3.0, 4.0)


In [6]:
try: 
    while True:
        print(next(v2))
except StopIteration as err:
    print(err)

3.0
4.0
There are only two elements


In [7]:
abs(v2)

5.0

In [8]:
bool(v2)

True

##### \_\_format\_\_ <a name='format'></a>
用来支持format(my_obj, format_spec)

In [9]:
format(v2, '^10.3fp') # 调用__format__方法,如果没定义则继承__str__方法

'<  5.000   ,   0.927   >'

In [10]:
format(v2, '.3e')

'(3.000e+00, 4.000e+00)'

## 用bytes来重建

In [11]:
c = bytes(v2) # 调用__bytes__
c

b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'

In [12]:
a = Array2D.frombytes(c)
a == v2 # 调用__eq__

True

In [13]:
a is v2 # compare id 

False

In [14]:
a

Array2D(3.000, 4.000)

In [15]:
hash(a)

7

In [16]:
type(a)

__main__.Array2D